In [1]:
Import-Module AWS.Tools.Common
Import-Module AWS.Tools.DynamoDBv2
Import-Module AWS.Tools.RDS

In [2]:
$RegexGroup             = 'AccountNumber'
$RegexGroupAccountName  = 'AccountName'
$RegexAccountNumber     = "(?<$RegexGroup>\d+):db.+$"
$RegexAccountNumberDDb  = "(?<$RegexGroup>\d+):table.+$" 
$RegexAccountName       = "(?<PreFix>DevOps-)(?<$RegexGroupAccountName>.+)"
$Profiles               = Get-AWSCredentials -ListProfileDetail | Where-Object { $_.ProfileName -match 'DevOps-Apex-PROD' }
$Regions                = "eu-central-1" , "eu-west-2" , "eu-west-1", "us-east-1"

function Get-AWSDBInformation {
    [CmdletBinding()]
    param (
        $Profiles,
        $Regions
    )
    
    begin {

        $Summary    = @()
        
    }
    
    process {

        $Summary    = @()
        foreach ($Profile in $Profiles) {

            foreach ($Region in $Regions) {
                
                $RDSInstances   = Get-RDSDBInstance -ProfileName $Profile.ProfileName -Region $Region
                $ddbTables      = Get-DDBTableList -ProfileName $Profile.ProfileName -Region $Region

                switch ([string]::IsNullOrEmpty($RDSInstances)) {
                    $true { 
                        if ($ddbTables) {
                            
                            $Table = $ddbTables | Select-Object -First 1
                            $ddbTable = Get-DDBTable -TableName $Table -ProfileName $Profile.ProfileName -Region $Region
                            $AccountID  = (($ddbTable.TableArn | Select-String -Pattern $RegexAccountNumberDDb).Matches.Groups | Where-Object { $_.Name -eq $RegexGroup }).Value
                        }else {

                            $AccountID = 0
                        }
                    }
                    $false {

                        $RDSInstance   = $RDSInstances | Select-Object -First 1
                        $AccountID  = (($RDSInstance.DBInstanceArn | Select-String -Pattern $RegexAccountNumber).Matches.Groups | Where-Object { $_.Name -eq $RegexGroup }).Value
                    }
                    default {

                        $AccountID = 0
                    }
                }

                $Sum = [PSCustomObject]@{

                    Account             = (($Profile.ProfileName | select-string -Pattern $RegexAccountName).Matches.Groups | Where-Object { $_.Name -eq $RegexGroupAccountName }).Value
                    AccountID           = $AccountID
                    Region              = $Region
                    RDSInstances        = ($RDSInstances.DBInstanceIdentifier | Measure-Object).Count
                    DynamoDBTables      = $ddbTables.Count
                    
                }

                $condition = $Sum.RDSInstances + $Sum.DynamoDBTables
                if ($condition -gt 0) {
                    $Summary += $Sum
                }
            }

        }
        
    }
    
    end {

        $Summary
        
    }
}

$Summary    = Get-AWSDBInformation $Profiles $Regions
$Summary    | Format-Table -AutoSize


Account   AccountID    Region    RDSInstances DynamoDBTables
-------   ---------    ------    ------------ --------------
Apex-PROD 463854377715 eu-west-2            0              8



In [3]:
$ddbTableList = Get-DDBTableList -ProfileName DevOps-Apex-PROD

$ddbTables  = @()
foreach ($ddbTable in $ddbTableList) {
    
    $Table = Get-DDBTable -TableName $ddbTable -ProfileName DevOps-Apex-PROD

    $Sum = [PSCustomObject]@{

        TableName           = $Table.TableName
        CreationDateTime    = $Table.CreationDateTime
        ItemCount           = $Table.ItemCount
        TableSizeBytes      = $Table.TableSizeBytes
        TableSizeMB         = "$([math]::round($($Table.TableSizeBytes /1MB), 0)) MB"
        TableStatus         = $Table.TableStatus
        #TableId             = $Table.TableId
        # KeySchema           = $Table.KeySchema
        # GlobalSecondaryIndexes = $Table.GlobalSecondaryIndexes
        
    }
    $ddbTables += $Sum
}
$ddbTables | Format-Table -AutoSize


TableName                  CreationDateTime      ItemCount TableSizeBytes TableSizeMB TableStatus
---------                  ----------------      --------- -------------- ----------- -----------
prod-accommodation         10/20/2023 1:11:55 PM    151530       29060289 28 MB       ACTIVE
prod-apex_product_extras   2/2/2023 4:43:28 PM     5071428     1041059031 993 MB      ACTIVE
prod-apex_translations     5/16/2024 10:06:42 AM      7743         735797 1 MB        ACTIVE
prod-availability          10/20/2023 1:11:55 PM   3882740     2743664869 2617 MB     ACTIVE
prod-bookability           10/20/2023 1:11:55 PM 117111222    32523228285 31017 MB    ACTIVE
prod-novasol_currencies    2/2/2023 4:43:28 PM          38            498 0 MB        ACTIVE
prod-novasol_product_cache 10/20/2023 1:11:55 PM    213274      203057925 194 MB      ACTIVE
terraform-locks            7/8/2022 6:20:39 PM          89          12684 0 MB        ACTIVE

